In [6]:
!pip install tesseract
!pip install tesseract-ocr

!sudo apt install tesseract-ocr
!pip install pytesseract

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 MB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tesseract: filename=tesseract-0.1.3-py3-none-any.whl size=45562552 sha256=dfdd1fb50c64dd8de6a6e851bc5cc1358007afb4e7826048b2575aa4f165a21f
  Stored in directory: /root/.cache/pip/wheels/71/c9/aa/698c579693e83fdda9ad6d6f0d8f61ed986e27925ef576f109
Successfully built tesseract
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tesseract-ocr
  Running setup.py clean for tesseract-ocr
Failed to build tesseract-ocr
ERROR: Could not build wheels for tesseract-ocr, which is required to install pyproject.toml-based projects
Reading package lists... Done
Building dependency tree... Done
Reading state in

In [81]:
!unzip /content/drive/MyDrive/Doctor_Bills.zip

Archive:  /content/drive/MyDrive/Doctor_Bills.zip
   creating: Doctor_Bills/
   creating: Doctor_Bills/test/
  inflating: Doctor_Bills/test/genuine-type01a-001.png  
  inflating: Doctor_Bills/test/genuine-type01a-024.png  
  inflating: Doctor_Bills/test/genuine-type01a-029.png  
  inflating: Doctor_Bills/test/genuine-type01b-007.png  
  inflating: Doctor_Bills/test/genuine-type01b-012.png  
  inflating: Doctor_Bills/test/genuine-type01b-020.png  
  inflating: Doctor_Bills/test/genuine-type02-006.png  
  inflating: Doctor_Bills/test/genuine-type02-008.png  
  inflating: Doctor_Bills/test/genuine-type02-039.png  
   creating: Doctor_Bills/test_bb/
   creating: Doctor_Bills/test_bm/
   creating: Doctor_Bills/train/
  inflating: Doctor_Bills/train/genuine-type01a-002.png  
  inflating: Doctor_Bills/train/genuine-type01a-003.png  
  inflating: Doctor_Bills/train/genuine-type01a-004.png  
  inflating: Doctor_Bills/train/genuine-type01a-005.png  
  inflating: Doctor_Bills/train/genuine-type01

In [8]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = (
    r'/usr/bin/tesseract'
)

In [ ]:
# from PIL import Image
# import pytesseract

# def extract_text_from_image(image_path):
#     image = Image.open(image_path)
#     text = pytesseract.image_to_string(image)
#     return text

# # Example usage
# image_path = '/content/genuine-type01a-001.png'
# extracted_text = extract_text_from_image(image_path)
# print("Extracted Text:", extracted_text)


# from fontTools.ttLib import TTFont
# from fontTools.unicode import Unicode
# import os

# def get_font_name(font_path):
#     font = TTFont(font_path)
#     for record in font['name'].names:
#         if record.nameID == 1 and record.platformID == 3:
#             return record.string.decode('utf-16-be')
#     return None

# def find_matching_fonts(extracted_text, fonts_directory):
#     matching_fonts = []
#     for font_file in os.listdir(fonts_directory):
#         if font_file.endswith('.ttf') or font_file.endswith('.otf'):
#             font_path = os.path.join(fonts_directory, font_file)
#             font_name = get_font_name(font_path)
#             if font_name:
#                 matching_fonts.append(font_name)
#     return matching_fonts

# # Example usage
# fonts_directory = '/content/Fonts'
# matching_fonts = find_matching_fonts(extracted_text, fonts_directory)
# print("Matching Fonts:", matching_fonts)


In [95]:
!rm -r /content/Doctor_Bills/test/test_bb

In [82]:
from PIL import Image
import os

def crop_image(image_path, crop_top, crop_left, crop_bottom, crop_right):
    """
    Crop an image using the specified dimensions and save it with the same name.

    Args:
    - image_path (str): Path to the original image file.
    - crop_top (int): Number of pixels to crop from the top.
    - crop_left (int): Number of pixels to crop from the left.
    - crop_bottom (int): Number of pixels to crop from the bottom.
    - crop_right (int): Number of pixels to crop from the right.
    """
    # Open the original image
    original_image = Image.open(image_path)

    # Get the dimensions of the original image
    width, height = original_image.size

    # Calculate the coordinates for cropping
    left = crop_left
    upper = crop_top
    right = width - crop_right   # Subtracting crop_right pixels from the width
    lower = height - crop_bottom  # Subtracting crop_bottom pixels from the height

    # Crop the image to the defined dimensions
    cropped_image = original_image.crop((left, upper, right, lower))

    # Save the cropped image with the same filename
    cropped_image.save(image_path)

# Function to iterate through a directory and apply cropping to each image
def crop_images_in_directory(directory):
    # Define cropping dimensions
    crop_top = 370
    crop_left = 220
    crop_bottom = 370
    crop_right = 210

    # Iterate through each file in the directory
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg')):  # Adjust file extensions as needed
                file_path = os.path.join(root, file)
                crop_image(file_path, crop_top, crop_left, crop_bottom, crop_right)

# Paths to the main directories containing train, test, val folders
main_folder = '/content/Doctor_Bills/'  # Replace with your main folder path

# Apply cropping to train, test, and val folders
train_folder = os.path.join(main_folder, 'train')
test_folder = os.path.join(main_folder, 'test')
val_folder = os.path.join(main_folder, 'val')

crop_images_in_directory(train_folder)
crop_images_in_directory(test_folder)
crop_images_in_directory(val_folder)


In [98]:
from PIL import Image, ImageDraw, ImageFont
import pytesseract
import random
import string
import os

# Configure tesseract command if needed
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def extract_text_with_boxes(image_path):
    image = Image.open(image_path)
    data = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)
    boxes = []
    for i in range(len(data['text'])):
        if int(data['conf'][i]) > 0:
            text = data['text'][i]
            x, y, w, h = data['left'][i], data['top'][i], data['width'][i], data['height'][i]
            boxes.append((text, (x, y, w, h)))
    return boxes

def get_font_name(font_path):
    font = TTFont(font_path)
    for record in font['name'].names:
        if record.nameID == 1 and record.platformID == 3:
            return record.string.decode('utf-16-be')
        elif record.nameID == 1 and record.platformID == 1:
            return record.string.decode('latin-1')
    return None

def find_matching_fonts(extracted_text, fonts_directory):
    matching_fonts = []
    for font_file in os.listdir(fonts_directory):
        if font_file.endswith('.ttf') or font_file.endswith('.otf'):
            font_path = os.path.join(fonts_directory, font_file)
            font_name = get_font_name(font_path)
            if font_name:
                matching_fonts.append((font_name, font_path))
    return matching_fonts

def find_font_size(image_path, box, font_path):
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    (x, y, w, h) = box
    font_size = 1
    font = ImageFont.truetype(font_path, font_size)

    # Increase font size until it fits within the bounding box
    while font.getsize('A')[1] < h:
        font_size += 1
        font = ImageFont.truetype(font_path, font_size)

    return font_size - 1  # Subtract 1 to ensure the font fits within the bounding box

def scramble_text(text):
    # Only scramble alphanumeric characters (letters and digits)
    characters = list(text)
    alphanum_indices = [idx for idx, char in enumerate(characters) if char.isalnum()]
    random.shuffle(alphanum_indices)
    scrambled_text = list(text)
    for i, idx in enumerate(alphanum_indices):
        scrambled_text[idx] = characters[i]
    return ''.join(scrambled_text)

def add_text_to_image(image, text, box, font_path, font_size):
    draw = ImageDraw.Draw(image)
    font = ImageFont.truetype(font_path, font_size)
    (x, y, w, h) = box

    # Erase the old text by drawing a white rectangle over it
    draw.rectangle([x, y, x + w, y + h], fill="white")

    # Add new text to the bounding box
    draw.text((x, y), text, font=font, fill='black')

def create_binary_mask(image_size, boxes, modified_boxes):
    mask = Image.new('1', image_size, color=0)  # Binary mask (0: black, 1: white)
    draw = ImageDraw.Draw(mask)

    for _, box in boxes:
        if box in modified_boxes:
            (x, y, w, h) = box
            draw.rectangle([x, y, x + w, y + h], fill=1)  # Mark the modified area as white
        else:
            (x, y, w, h) = box
            draw.rectangle([x, y, x + w, y + h], fill=0)  # Mark the unmodified area as black

    return mask

def apply_forgery_and_save(image_path, fonts_directory, output_folder):
    # Extract text and their bounding boxes
    boxes = extract_text_with_boxes(image_path)

    # Find matching fonts (this should be done in a more sophisticated way in a real application)
    matching_fonts = find_matching_fonts([text for text, _ in boxes], fonts_directory)

    if boxes and matching_fonts:
        total_bounding_boxes = len(boxes)

        # Create a copy of the original image to work on
        modified_image = Image.open(image_path).copy()

        # Select 30% of the bounding boxes randomly
        num_boxes_to_modify = max(1, len(boxes) * 3 // 10)  # 30% of total bounding boxes
        selected_boxes = random.sample(boxes, num_boxes_to_modify)
        modified_boxes = []

        for selected_box in selected_boxes:
            text_in_box, box = selected_box

            # Get the font path for the text in the selected bounding box
            font_path = matching_fonts[0][1]  # Assuming the first matching font is the right one

            # Find the font size that fits in the bounding box
            font_size = find_font_size(image_path, box, font_path)

            # Scramble the original text
            scrambled_text = scramble_text(text_in_box)

            # Add new text (scrambled) to the bounding box in the modified image
            add_text_to_image(modified_image, scrambled_text, box, font_path, font_size)

            # Record modified box for the binary mask
            modified_boxes.append(box)

        # Determine if the image has forgeries (modifications)
        has_forgery = len(modified_boxes) > 0

        # Save the modified image to the output folder
        output_image_folder = os.path.join(output_folder, '_bb')
        os.makedirs(output_image_folder, exist_ok=True)
        modified_image.save(os.path.join(output_image_folder, os.path.basename(image_path)))

        # Create and save the binary mask
        binary_mask = create_binary_mask(modified_image.size, boxes, modified_boxes)
        binary_mask_folder = os.path.join(output_folder, '_bm')
        os.makedirs(binary_mask_folder, exist_ok=True)
        binary_mask.save(os.path.join(binary_mask_folder, os.path.basename(image_path)))

        return True, total_bounding_boxes, num_boxes_to_modify
    else:
        return False, 0, 0

# Example usage for train, val, test folders
main_folder = '/content/Doctor_Bills/'  # Replace with your main folder path
selected_image_paths = train_images

fonts_directory = '/content/Fonts'  # Replace with your fonts directory

for image_path in selected_image_paths:
    # output_folder = os.path.dirname(image_path)  # Output folder will be the same as original image folder
    output_folder = '/content/Doctor_Bills/train'
    success, total_bounding_boxes, num_boxes_to_modify = apply_forgery_and_save(image_path, fonts_directory, output_folder)
    if success:
        print(f"Successfully modified {image_path}:")
        print(f"Total bounding boxes found in the image: {total_bounding_boxes}")
        print(f"Number of bounding boxes modified: {num_boxes_to_modify}")
        print("============================================")
    else:
        print(f"No text bounding boxes found in {image_path}. Skipping...")
        print("============================================")


<ipython-input-98-cfecd45feffc>:48: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  while font.getsize('A')[1] < h:


Successfully modified /content/Doctor_Bills/train/genuine-type01a-023.png:
Total bounding boxes found in the image: 264
Number of bounding boxes modified: 79
Successfully modified /content/Doctor_Bills/train/genuine-type02-004.png:
Total bounding boxes found in the image: 151
Number of bounding boxes modified: 45
Successfully modified /content/Doctor_Bills/train/genuine-type02-038.png:
Total bounding boxes found in the image: 150
Number of bounding boxes modified: 45
Successfully modified /content/Doctor_Bills/train/genuine-type02-028.png:
Total bounding boxes found in the image: 152
Number of bounding boxes modified: 45
Successfully modified /content/Doctor_Bills/train/genuine-type02-016.png:
Total bounding boxes found in the image: 151
Number of bounding boxes modified: 45
Successfully modified /content/Doctor_Bills/train/genuine-type01b-004.png:
Total bounding boxes found in the image: 294
Number of bounding boxes modified: 88
Successfully modified /content/Doctor_Bills/train/genuin

In [90]:
import os

def count_files_in_folder(folder_path):
    # Initialize a counter for files
    file_count = 0

    # Iterate through all files in the folder
    for _, _, files in os.walk(folder_path):
        file_count += len(files)

    return file_count

# Example usage:
folder_path = '/content/Doctor_Bills/train'  # Replace with your folder path
num_files = count_files_in_folder(folder_path)
print(f"Number of files in '{folder_path}': {num_files}")


Number of files in '/content/Doctor_Bills/train': 72


In [92]:
import os
import random

# Function to select 30% of images from a folder
def select_30_percent_images(folder_path):
    # List all files in the folder
    all_files = os.listdir(folder_path)

    # Filter image files (you can add more extensions if needed)
    image_files = [file for file in all_files if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Calculate number of files to select (30% of total)
    num_files_to_select = int(len(image_files) / 3)
    print(num_files_to_select)

    # Randomly select 30% of images
    selected_images = random.sample(image_files, num_files_to_select)

    # Create list of full file paths
    selected_image_paths = [os.path.join(folder_path, image) for image in selected_images]

    return selected_image_paths

# Example usage
folder_path = '/content/Doctor_Bills/train'  # Replace with your folder path
train_images = select_30_percent_images(folder_path)

folder_path = '/content/Doctor_Bills/test'  # Replace with your folder path
test_images = select_30_percent_images(folder_path)

folder_path = '/content/Doctor_Bills/val'  # Replace with your folder path
val_images = select_30_percent_images(folder_path)

train_images

# # Print the list of selected image paths
# for image_path in selected_images:
#     print(image_path)


24
3
3


['/content/Doctor_Bills/train/genuine-type01a-023.png',
 '/content/Doctor_Bills/train/genuine-type02-004.png',
 '/content/Doctor_Bills/train/genuine-type02-038.png',
 '/content/Doctor_Bills/train/genuine-type02-028.png',
 '/content/Doctor_Bills/train/genuine-type02-016.png',
 '/content/Doctor_Bills/train/genuine-type01b-004.png',
 '/content/Doctor_Bills/train/genuine-type01b-008.png',
 '/content/Doctor_Bills/train/genuine-type02-026.png',
 '/content/Doctor_Bills/train/genuine-type01a-005.png',
 '/content/Doctor_Bills/train/genuine-type01b-001.png',
 '/content/Doctor_Bills/train/genuine-type01b-018.png',
 '/content/Doctor_Bills/train/genuine-type02-023.png',
 '/content/Doctor_Bills/train/genuine-type01a-007.png',
 '/content/Doctor_Bills/train/genuine-type01a-011.png',
 '/content/Doctor_Bills/train/genuine-type01a-020.png',
 '/content/Doctor_Bills/train/genuine-type01a-002.png',
 '/content/Doctor_Bills/train/genuine-type02-013.png',
 '/content/Doctor_Bills/train/genuine-type01a-016.png',

In [105]:
from PIL import Image, ImageDraw
import os

def create_black_mask(image_path, mask_folder):
    image = Image.open(image_path)
    mask = Image.new('1', image.size, color=0)  # Binary mask (0: black)
    mask.save(os.path.join(mask_folder, os.path.basename(image_path)))

def copy_image_with_mask(image_path, destination_folder, mask_folder):
    image_name = os.path.basename(image_path)
    image = Image.open(image_path)
    image.save(os.path.join(destination_folder, image_name))
    create_black_mask(image_path, mask_folder)

def main(source_folder, destination_folder, mask_folder, image_names_to_keep):
    os.makedirs(destination_folder, exist_ok=True)
    os.makedirs(mask_folder, exist_ok=True)

    for root, _, files in os.walk(source_folder):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):  # Adjust file extensions as needed
                image_path = os.path.join(root, file)
                if os.path.basename(image_path) not in image_names_to_keep:
                    copy_image_with_mask(image_path, destination_folder, mask_folder)

# Example usage
source_folder = '/content/Doctor_Bills/val'
destination_folder = '/content/Doctor_Bills/val_bb'
mask_folder = '/content/Doctor_Bills/val_bm'
image_names_to_keep = val_images  # List of image names to keep

main(source_folder, destination_folder, mask_folder, image_names_to_keep)


# Example usage
source_folder = '/content/Doctor_Bills/train'
destination_folder = '/content/Doctor_Bills/train_bb'
mask_folder = '/content/Doctor_Bills/train_bm'
image_names_to_keep = train_images  # List of image names to keep

main(source_folder, destination_folder, mask_folder, image_names_to_keep)


In [104]:
test_images

['/content/Doctor_Bills/test/genuine-type02-039.png',
 '/content/Doctor_Bills/test/genuine-type02-008.png',
 '/content/Doctor_Bills/test/genuine-type01b-012.png']

In [102]:
!rm -r /content/Doctor_Bills/test/_bb
!rm -r /content/Doctor_Bills/test/_bm

!rm -r /content/Doctor_Bills/val/_bb
!rm -r /content/Doctor_Bills/val/_bm

!rm -r /content/Doctor_Bills/train/_bb
!rm -r /content/Doctor_Bills/train/_bm

In [101]:
import os
import shutil

def move_images(source_folder, destination_folder):
    # Create destination folder if it doesn't exist
    os.makedirs(destination_folder, exist_ok=True)

    # Iterate over files in source folder
    for file_name in os.listdir(source_folder):
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            source_path = os.path.join(source_folder, file_name)
            destination_path = os.path.join(destination_folder, file_name)
            shutil.move(source_path, destination_path)
            print(f"Moved {file_name} to {destination_folder}")

# Example usage
source_folder = '/content/Doctor_Bills/train/_bb'
destination_folder = '/content/Doctor_Bills/train_bb'

move_images(source_folder, destination_folder)

source_folder = '/content/Doctor_Bills/train/_bm'
destination_folder = '/content/Doctor_Bills/train_bm'

move_images(source_folder, destination_folder)


Moved genuine-type01a-011.png to /content/Doctor_Bills/train_bb
Moved genuine-type02-016.png to /content/Doctor_Bills/train_bb
Moved genuine-type01a-025.png to /content/Doctor_Bills/train_bb
Moved genuine-type02-028.png to /content/Doctor_Bills/train_bb
Moved genuine-type01a-005.png to /content/Doctor_Bills/train_bb
Moved genuine-type01b-018.png to /content/Doctor_Bills/train_bb
Moved genuine-type01b-004.png to /content/Doctor_Bills/train_bb
Moved genuine-type01b-008.png to /content/Doctor_Bills/train_bb
Moved genuine-type01a-016.png to /content/Doctor_Bills/train_bb
Moved genuine-type01a-002.png to /content/Doctor_Bills/train_bb
Moved genuine-type01a-020.png to /content/Doctor_Bills/train_bb
Moved genuine-type02-038.png to /content/Doctor_Bills/train_bb
Moved genuine-type02-004.png to /content/Doctor_Bills/train_bb
Moved genuine-type02-026.png to /content/Doctor_Bills/train_bb
Moved genuine-type01b-001.png to /content/Doctor_Bills/train_bb
Moved genuine-type02-022.png to /content/Doct

In [108]:
import shutil
shutil.make_archive('doctor_bills', 'zip', '/content/Doctor_Bills')

'/content/doctor_bills.zip'

In [4]:
# !rm -r /content/test
# !rm -r /content/test_bm
# !rm -r /content/test_bb

# !rm -r /content/val
# !rm -r /content/val_bm
# !rm -r /content/val_bb

# !rm -r /content/train
# !rm -r /content/train_bm
# !rm -r /content/train_bb

# Split into Patches

In [ ]:
!mkdir doctor_bills
!unzip /content/drive/MyDrive/doctor_bills.zip -d /content/doctor_bills

In [10]:
!mkdir /content/doctor_bills/train_patches
!mkdir /content/doctor_bills/train_patches_masks
!mkdir /content/doctor_bills/train_patches_bm

!mkdir /content/doctor_bills/test_patches
!mkdir /content/doctor_bills/test_patches_masks
!mkdir /content/doctor_bills/test_patches_bm

!mkdir /content/doctor_bills/val_patches
!mkdir /content/doctor_bills/val_patches_masks
!mkdir /content/doctor_bills/val_patches_bm



mkdir: cannot create directory ‘/content/doctor_bills/train_patches’: File exists
mkdir: cannot create directory ‘/content/doctor_bills/train_patches_masks’: File exists
mkdir: cannot create directory ‘/content/doctor_bills/train_patches_bm’: File exists
mkdir: cannot create directory ‘/content/doctor_bills/test_patches’: File exists
mkdir: cannot create directory ‘/content/doctor_bills/test_patches_masks’: File exists
mkdir: cannot create directory ‘/content/doctor_bills/test_patches_bm’: File exists
mkdir: cannot create directory ‘/content/doctor_bills/val_patches_masks’: File exists
mkdir: cannot create directory ‘/content/doctor_bills/val_patches_bm’: File exists


In [9]:
!rm -r /content/doctor_bills/val_patches

In [21]:
import cv2
import os
import numpy as np
from PIL import Image

def resize_to_nearest_multiple(image, patch_size):
    """Resize image to the nearest dimensions that are multiples of patch_size."""
    h, w = image.shape[:2]
    new_h = (h // patch_size) * patch_size
    new_w = (w // patch_size) * patch_size
    resized_image = cv2.resize(image, (new_w, new_h))
    return resized_image

def split_into_patches(image, patch_size):
    """Split image into patches of size patch_size x patch_size."""
    patches = []
    h, w = image.shape[:2]
    for y in range(0, h, patch_size):
        for x in range(0, w, patch_size):
            patch = image[y:y+patch_size, x:x+patch_size]
            patches.append(patch)
    return patches

def save_patches(patches, output_dir, base_name):
    """Save the patches to the specified directory."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for idx, patch in enumerate(patches):
        patch_filename = f"{base_name}_patch_{idx}.png"
        cv2.imwrite(os.path.join(output_dir, patch_filename), patch)

def process_folder(input_folder, output_folder, patch_size=256):
    """Process each image in the folder, resize, split into patches, and save the patches."""
    for root, _, files in os.walk(input_folder):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg')):  # Adjust file extensions as needed
                image_path = os.path.join(root, file)
                # Load image
                image = cv2.imread(image_path)
                if image is not None:
                    # Resize image to the nearest multiple of 256
                    resized_image = resize_to_nearest_multiple(image, patch_size)
                    # Split the image into patches
                    patches = split_into_patches(resized_image, patch_size)
                    # Save the patches
                    save_patches(patches, output_folder, os.path.splitext(file)[0])
                    print(f"Processed {file}: saved {len(patches)} patches to {output_folder}")
                else:
                    print(f"Failed to load {file}")

# Example usage
# input_folder  = '/content/doctor_bills/train'
# output_folder = input_folder + '_patches'  # Replace with your output folder path

# input_folder  = '/content/doctor_bills/train_bb'
# output_folder = '/content/doctor_bills/train_patches_masks'  # Replace with your output folder path

input_folder  = '/content/doctor_bills/train_bm'
output_folder = '/content/doctor_bills/train_patches_bm'  # Replace with your output folder path

process_folder(input_folder, output_folder)


Processed genuine-type01a-019.png: saved 80 patches to /content/doctor_bills/train_patches_bm
Processed genuine-type02-034.png: saved 80 patches to /content/doctor_bills/train_patches_bm
Processed genuine-type02-018.png: saved 80 patches to /content/doctor_bills/train_patches_bm
Processed genuine-type02-010.png: saved 80 patches to /content/doctor_bills/train_patches_bm
Processed genuine-type01a-012.png: saved 80 patches to /content/doctor_bills/train_patches_bm
Processed genuine-type01b-008.png: saved 80 patches to /content/doctor_bills/train_patches_bm
Processed genuine-type02-019.png: saved 80 patches to /content/doctor_bills/train_patches_bm
Processed genuine-type01b-001.png: saved 80 patches to /content/doctor_bills/train_patches_bm
Processed genuine-type02-031.png: saved 80 patches to /content/doctor_bills/train_patches_bm
Processed genuine-type02-038.png: saved 80 patches to /content/doctor_bills/train_patches_bm
Processed genuine-type02-016.png: saved 80 patches to /content/doc

In [22]:
import os

def count_files_in_folder(folder_path):
    # Initialize a counter for files
    file_count = 0

    # Iterate through all files in the folder
    for _, _, files in os.walk(folder_path):
        file_count += len(files)

    return file_count

# Example usage:
folder_path = '/content/doctor_bills/train_patches'  # Replace with your folder path
num_files = count_files_in_folder(folder_path)
print(f"Number of files in '{folder_path}': {num_files}")

folder_path = '/content/doctor_bills/train_patches_masks'  # Replace with your folder path
num_files = count_files_in_folder(folder_path)
print(f"Number of files in '{folder_path}': {num_files}")

folder_path = '/content/doctor_bills/train_patches_bm'  # Replace with your folder path
num_files = count_files_in_folder(folder_path)
print(f"Number of files in '{folder_path}': {num_files}")


Number of files in '/content/doctor_bills/train_patches': 5760
Number of files in '/content/doctor_bills/train_patches_masks': 5760
Number of files in '/content/doctor_bills/train_patches_bm': 5760


In [23]:
!rm -r /content/doctor_bills/test
!rm -r /content/doctor_bills/test_bb
!rm -r /content/doctor_bills/test_bm

!rm -r /content/doctor_bills/train
!rm -r /content/doctor_bills/train_bb
!rm -r /content/doctor_bills/train_bm

!rm -r /content/doctor_bills/val
!rm -r /content/doctor_bills/val_bb
!rm -r /content/doctor_bills/val_bm

In [24]:
import shutil
shutil.make_archive('doctor_bills_patches', 'zip', '/content/doctor_bills')

'/content/doctor_bills_patches.zip'